### srtm dem processing  
including dem mosaic, downsampling and clipping.


In [1]:
import os
import rasterio as rio
import geopandas as gpd
from glob import glob
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling


### Mosaic

In [3]:
paths_dem_ls = glob('data/dem/tiles/*')
path_mosaic = 'data/dem/hma_SRTMGL3_90m.tif'

src_files_to_mosaic = []
for fp in paths_dem_ls:
    src = rio.open(fp)
    src_files_to_mosaic.append(src)
mosaic_arr, mosaic_trans = merge(src_files_to_mosaic)
mosaic_meta = src.meta.copy()
mosaic_meta.update({
    "height": mosaic_arr.shape[1],
    "width": mosaic_arr.shape[2],
    "transform": mosaic_trans
    })

# # Write the mosaic raster to disk
# with rio.open(path_mosaic, 'w', **mosaic_meta) as dest:
#     dest.write(mosaic_arr)


### Downsampling

In [3]:
path_srtm = 'data/dem/hma_SRTMGL3_90m.tif'
path_srtm_down = 'data/dem/hma_SRTMGL3_500m.tif' 

with rio.open(path_srtm) as src:
    scale_factor = 500 / 90  
    dst_width, dst_height = int(src.width / scale_factor), int(src.height / scale_factor)    
    dst_transform = src.transform * src.transform.scale(scale_factor, scale_factor)
    dst_meta = src.meta.copy()
    dst_meta.update({
        "width": dst_width,
        "height": dst_height,
        "transform": dst_transform})
    # with rio.open(path_srtm_down, "w", **dst_meta) as dst:
    #     reproject(
    #         source=rio.band(src, 1),
    #         destination=rio.band(dst, 1),
    #         src_transform=src.transform,
    #         src_crs=src.crs,
    #         dst_transform=dst_transform,
    #         dst_crs=src.crs,
    #         resampling=Resampling.average,  
    #     )


#### clip to hma subregions. 

In [5]:
path_dem = 'data/dem/hma_SRTMGL3_90m.tif'
path_hma_gtng = 'data/hma-extent/HMA/gtng_202307_hma_subregions.gpkg'
hma_vec_gdf = gpd.read_file(path_hma_gtng)

with rio.open(path_dem) as src:
    if hma_vec_gdf.crs != src.crs: 
        hma_vec_gdf = hma_vec_gdf.to_crs(src.crs)      
    for idx, row in hma_vec_gdf.iterrows():
        geom = row.geometry
        clipped_arr, clipped_transform = mask(dataset=src, shapes = [geom], 
                                              crop=True, all_touched=True)
        meta = src.meta.copy()
        meta.update({
            "height": clipped_arr.shape[1],
            "width": clipped_arr.shape[2],
            "transform": clipped_transform})
        # ## save to path
        # name_subregion = row['full_name'].split(' (')[0].replace(' ', '_').lower()
        # path_save = f"data/dem/hma-subregions/dem_90m_{name_subregion}.tif"
        # if os.path.exists(path_save): os.remove(path_save)
        # with rio.open(path_save, "w", **meta) as dst:
        #     print(dst.bounds)
        #     dst.write(clipped_arr)
        # print(f"saved to: {path_save}")
